<a href="https://colab.research.google.com/github/hamdankalyar/Amazon-Audible-Scraper/blob/main/AmazonAudioScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')


In [ ]:
!pip install chromedriver_autoinstaller

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller

In [9]:
pip install pandas

In [ ]:
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920,1080')  # Set window size here
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)


# Open the page
driver.get("https://www.audible.com/adblbestsellers?searchCategory=18571912011&ref_pageloadid=HjWkt5IHoHXlTkLl&ref=a_cat_Arts-_c4_showmore&pf_rd_p=d5744c47-5ad0-4c12-bb42-e6207b17068a&pf_rd_r=DX33H3JK4HJ1J4AWJ875&pageLoadId=8zZgXkl7m2NZkEES&creativeId=f96a42f6-d65d-454f-b2f9-5abd1c38a6d7")



try:
    items_list = driver.find_elements(By.XPATH, '//div[@data-widget="productList"]/div/span[2]/ul/li')

    book_title = []
    book_subtitle = []
    book_authors = []  # List to store authors
    book_prices = []  # List to store prices

    for counter, item in enumerate(items_list):
        title = item.get_attribute('aria-label')
        book_title.append(title)

        try:
            subtitle = item.find_element(By.XPATH, './/li[contains(@class, "subtitle")]/span').text
        except NoSuchElementException:
            subtitle = "No Subtitle"  # Handle missing subtitle
        book_subtitle.append(subtitle)

        # Finding the authors
        author_elements = item.find_elements(By.XPATH, './/li[contains(@class, "authorLabel")]/span/a')
        authors = [author.text for author in author_elements]
        book_authors.append(", ".join(authors))  # Join multiple authors with comma

        # Finding the price
        price = item.find_element(By.XPATH, './/div[contains(@class, "adblBuyBoxPrice")]/p[1]/span[2]').text
        book_prices.append(price)

        # Print book details for debugging
        print(f"Book {counter}: Title - {title}, Subtitle - {subtitle}, Authors - {', '.join(authors)}, Price - {price}")

    # Create DataFrame
    df_books = pd.DataFrame({
        'Title': book_title,
        'Subtitle': book_subtitle,
        'Authors': book_authors,
        'Price': book_prices
    })
    df_books.to_csv('books.csv', index=False)

    # Print all collected data for verification
    print(df_books)

except Exception as e:
    print("Element not found or error in scraping:", e)

finally:
    driver.quit()

In [ ]:
# this code is with the pagination concept

import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--window-size=1920,1080')  # Set window size here
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)


# Open the page
driver.get("https://www.audible.com/adblbestsellers?searchCategory=18571912011&ref_pageloadid=HjWkt5IHoHXlTkLl&ref=a_cat_Arts-_c4_showmore&pf_rd_p=d5744c47-5ad0-4c12-bb42-e6207b17068a&pf_rd_r=DX33H3JK4HJ1J4AWJ875&pageLoadId=8zZgXkl7m2NZkEES&creativeId=f96a42f6-d65d-454f-b2f9-5abd1c38a6d7")

# Locate pagination
pagination = driver.find_element(By.XPATH, '//ul[contains(@class, "pagingElements")]')
pages = pagination.find_elements(By.TAG_NAME, 'li')
last_page = int(pages[-2].text)  # Assuming the second last element is the last page number

book_title = []
book_subtitle = []
book_authors = []
book_prices = []

current_page = 1
while current_page <= last_page:
    items_list = driver.find_elements(By.XPATH, '//div[@data-widget="productList"]/div/span[2]/ul/li')
    for counter, item in enumerate(items_list):
        title = item.get_attribute('aria-label')
        book_title.append(title)

        try:
            subtitle = item.find_element(By.XPATH, './/li[contains(@class, "subtitle")]/span').text
        except NoSuchElementException:
            subtitle = "No Subtitle"
        book_subtitle.append(subtitle)

        author_elements = item.find_elements(By.XPATH, './/li[contains(@class, "authorLabel")]/span/a')
        authors = [author.text for author in author_elements]
        book_authors.append(", ".join(authors))

        try:
            price = item.find_element(By.XPATH, './/div[contains(@class, "adblBuyBoxPrice")]/p[1]/span[2]').text
        except NoSuchElementException:
            price = "Price Unavailable"
        book_prices.append(price)

    # After processing all items on the current page
    if current_page < last_page:
        try:
            next_page = driver.find_element(By.XPATH, './/span[contains(@class , "nextButton")]')
            next_page.click()
            time.sleep(2)  # Ensure the page has loaded
        except NoSuchElementException:
            print(f"Failed to locate next page button on page {current_page}.")
            break
    current_page += 1

# Close the driver
driver.quit()

# Save to DataFrame and CSV
df_books = pd.DataFrame({
    'Title': book_title,
    'Subtitle': book_subtitle,
    'Authors': book_authors,
    'Price': book_prices
})
df_books.to_csv('books.csv', index=False)
print(df_books)